In [1]:
from modules.oracle_connection import *
from queries.queries import *
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

###########################################################################################################
###########################################################################################################

conn_string_suc = {
        'user': 'CONSULTA_WF',
        'password': 'Coto__2024',
        'host': 'wf',
        'port': 1521,
        's_name': 'wf009a'
}


def retrieve_oracle_data(conn_string, query):
    
    # Establezco la conexion a la base de datos Oracle
    DDBB = Oracle_Conection(**conn_string)
    DDBB.Connect()
    
    return pd.read_sql(query, con=DDBB.conn)

In [2]:
# Establezco los parametros de las consultas
output_file = './data/{filename}.csv'

# df_pallets = retrieve_oracle_data(conn_string_suc, Q_PALLETS)
# df_pallets.head()



df_retrabajos = retrieve_oracle_data(conn_string_suc, Q_RETRABAJOS)
df_retrabajos.head()



# # Extraigo la base de resumen
# df_resumen = retrieve_oracle_data(conn_string_suc, Q_RESUMEN.format(fmin=fmin, fmax=fmax))
# filename = 'df_resumen_202404'
# df_resumen.to_csv(output_file.format(sucursal=sucursal, filename=filename), index=False, decimal=',', sep=';')


,OC,REMITO,DESCRIPCIONRETRABAJO,CANTIDAD,CODPROVEEDOR,PROVEEDOR,FECHAENTRADA,REMITOGDM,COSTO,FECHAPROCESAMIENTOGDM
0,20484870093,002700008132,DESCARGA TRASERA,9,13045,DULCOR S.A.,15/10/2024,372449,50692.59,20/10/2024
1,20484870093,002700008132,REPALETIZACION,4,13045,DULCOR S.A.,15/10/2024,372449,45060.12,20/10/2024


In [4]:
df_rechazos = retrieve_oracle_data(conn_string_suc, Q_RECHAZOS)
df_rechazos.head()

,ID,IDCABECERA,PLU,DESCPLU,OC,CODIGOPROVEEDOR,DESCPROVEEDOR,UNIDADMEDIDA,CANTIDADENTREGADA,CANTIDADRECHAZADA,OBSERVACIONES,IDMOTIVO,DESCMOTIVO,LEGAJOINSPECTOR,INSPECTOR,VL,CORTE
0,27,386601,691,ZAPALLITO REDONDO XKG 1 KGM,13667516093,177289,PRODUCTOS AMERICO S.R.L.,BUL,73.0,73.0,None,9,Plagas,148285,None,1,18
1,39,389690,691,ZAPALLITO REDONDO XKG 1 KGM,13692597093,177289,PRODUCTOS AMERICO S.R.L.,BUL,112.0,112.0,None,1,Averías,109218,None,2,16
2,26,386601,593,BERENJENA N. INVERNACULO XKG,13667516093,177289,PRODUCTOS AMERICO S.R.L.,BUL,26.0,26.0,None,1,Averías,148285,None,2,10
3,46,396934,684,TOMATE RED . XKG 1 KGM,13740792093,177289,PRODUCTOS AMERICO S.R.L.,BUL,128.0,128.0,Manchados\n,1,Averías,72513,RIVERO VICTOR MARIO,5,18
4,115,575472,28736,MUZZARELLA ZONA E! . X 1 KGM,51546194093,200579,LACTEOS BARRAZA SA,BUL,150.0,75.0,Vto proximo durabilidad 90 dias entrega con 47...,6,Vencimiento,17047,CICINIO MONICA ELIZABETH,4,14


In [19]:
cols_index = ['IDCABECERA','DESCPROVEEDOR','DESCMOTIVO']
df_rechazos_pi = df_rechazos.groupby(cols_index).agg({'CANTIDADENTREGADA':sum,'CANTIDADRECHAZADA':sum}).reset_index()
df_rechazos_pi.head(10)

,IDCABECERA,DESCPROVEEDOR,DESCMOTIVO,CANTIDADENTREGADA,CANTIDADRECHAZADA
0,284349,DOLE NAT CO. S.A,Averías,10.0,1.0
1,284349,DOLE NAT CO. S.A,Deshidratacion,1.0,1.0
2,353436,AVICOLA BRANDSEN S.A.,Deshidratacion,700.0,100.0
3,353436,AVICOLA BRANDSEN S.A.,Madurez,250.0,100.0
4,354018,TRES ASES S.A,Averías,167.0,167.0
5,355048,ALMANA S.R.L.,Averías,220.0,220.0
6,355441,MARIO CERVI e HIJOS S.A.C.I.A.,Tamaño,250.0,250.0
7,355473,CAMPO LAVALLE SA,Averías,22000.0,22000.0
8,355596,EL RESGUARDO S.A.,Averías,400.0,400.0
9,357443,FRAGAPANE HERMANOS S.R.L.,Averías,82.0,82.0


In [10]:
df_flujo = retrieve_oracle_data(conn_string_suc, Q_FLUJO)
df_flujo.head()

,ID_TIPOPI,PI,PI_DESC,ID_ESTADO,ESTADO,FECHA,ORDEN
0,2,26610,RECEPCION SECOS,25,PERMISO ANULADO,2024-07-02 08:40:46,1
1,2,142775,RECEPCION SECOS,25,PERMISO ANULADO,2024-02-21 16:31:24,1
2,2,1407774,RECEPCION SECOS,15,PENDIENTE DE ARRIBO,2024-01-03 10:53:20,1
3,2,1407774,RECEPCION SECOS,16,ARRIBADO AL PREDIO,2024-01-03 10:54:07,2
4,2,1407774,RECEPCION SECOS,17,CON ANDEN ASIGNADO,2024-01-03 10:59:15,3


In [18]:
mask_pi = df_flujo['PI']==386601
df_flujo[mask_pi].head()

,ID_TIPOPI,PI,PI_DESC,ID_ESTADO,ESTADO,FECHA,ORDEN


In [17]:
df_rechazos.sort_values(by='IDCABECERA', ascending=False)
df_rechazos.head(10)

,ID,IDCABECERA,PLU,DESCPLU,OC,CODIGOPROVEEDOR,DESCPROVEEDOR,UNIDADMEDIDA,CANTIDADENTREGADA,CANTIDADRECHAZADA,OBSERVACIONES,IDMOTIVO,DESCMOTIVO,LEGAJOINSPECTOR,INSPECTOR,VL,CORTE
0,27,386601,691,ZAPALLITO REDONDO XKG 1 KGM,13667516093,177289,PRODUCTOS AMERICO S.R.L.,BUL,73.0,73.0,None,9,Plagas,148285,None,1,18
1,39,389690,691,ZAPALLITO REDONDO XKG 1 KGM,13692597093,177289,PRODUCTOS AMERICO S.R.L.,BUL,112.0,112.0,None,1,Averías,109218,None,2,16
2,26,386601,593,BERENJENA N. INVERNACULO XKG,13667516093,177289,PRODUCTOS AMERICO S.R.L.,BUL,26.0,26.0,None,1,Averías,148285,None,2,10
3,46,396934,684,TOMATE RED . XKG 1 KGM,13740792093,177289,PRODUCTOS AMERICO S.R.L.,BUL,128.0,128.0,Manchados\n,1,Averías,72513,RIVERO VICTOR MARIO,5,18
4,115,575472,28736,MUZZARELLA ZONA E! . X 1 KGM,51546194093,200579,LACTEOS BARRAZA SA,BUL,150.0,75.0,Vto proximo durabilidad 90 dias entrega con 47...,6,Vencimiento,17047,CICINIO MONICA ELIZABETH,4,14
5,103,571253,28736,MUZZARELLA ZONA E! . X 1 KGM,14954590093,200579,LACTEOS BARRAZA SA,BUL,150.0,150.0,Vto entregado igual 20-11 e inferior 16-11 al ...,6,Vencimiento,17047,CICINIO MONICA ELIZABETH,4,14
6,12,355473,456,SANDIA NACI0NAL . 1 KGM,13451779093,178627,CAMPO LAVALLE SA,BUL,22000.0,22000.0,Golpes averías bins de maderas rotos,1,Averías,148285,None,7,9
7,18,358411,861,UVA RED GLOBE . 1 KGM,13482021093,198947,ZOBERANO SA,BUL,320.0,320.0,Podredumbre,1,Averías,148285,None,4,12
8,58,538041,22866,HUEVO BLANCO EXTRA . BLI 6 U,14720876093,180529,AVICOLA TELOS S A,BUL,42.0,42.0,Se detectan unidades con venc poco legible,20,Novedad de entrega,17047,CICINIO MONICA ELIZABETH,2,30
9,84,560966,22875,HUEVO COLOR . CJA 6 UNI,14893376093,180529,AVICOLA TELOS S A,BUL,96.0,1.0,Vencimeinto 29/09 resto 2/10,20,Novedad de entrega,17047,CICINIO MONICA ELIZABETH,2,30


In [7]:
Q_PI = """SELECT * FROM pi_cabecera"""
df_PI = retrieve_oracle_data(conn_string_suc, Q_PI)
df_PI.head(50)

,ID,ACTIVO,STAMP,VERSION,CODIGOBARRAS,OBSERVACIONES,PATENTESEMI,PATENTETRACTOR,RAZONSOCIAL,SECUENCIA,VIGENCIADESDE,VIGENCIAHASTA,ID_ESTADO,ID_SISTEMAORIGEN,ID_TIPOPI,ID_ULTIMOMOVIMIENTO,PIPRINCIPAL_ID,ID_TIPOMOVIL,CODIGO_TARJETA
0,35,S,2012-04-30 17:12:46,0,PI00000000035,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,"ORBIS MERTIG S.A.C.I.,",0,2012-05-02 06:00:00,2012-05-03 06:00:00,25,90,3,7417772,NaN,0,None
1,36,S,2012-04-30 17:12:46,0,PI00000000036,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,GE CONSUMER INDUSTRIAL ARGEN,0,2012-05-02 06:00:00,2012-05-03 06:00:00,25,90,3,7417774,NaN,0,None
2,37,S,2012-04-30 17:12:46,0,PI00000000037,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,WHIRLPOOL ARGENTINA S A,0,2012-05-02 06:00:00,2012-05-03 06:00:00,25,90,3,7417773,NaN,0,None
3,38,S,2012-04-30 17:12:47,0,PI00000000038,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,LILIANA S.R.L.,0,2012-05-02 07:00:00,2012-05-03 07:00:00,25,90,3,7417775,NaN,0,None
4,39,S,2012-04-30 17:12:47,0,PI00000000039,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,3 M ARGENTINA S.A.C.I.F.I.A.,0,2012-05-02 07:00:00,2012-05-03 07:00:00,25,90,3,7417776,NaN,0,None
5,40,S,2012-04-30 17:12:47,0,PI00000000040,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,EMAPI SA,0,2012-05-02 07:00:00,2012-05-03 07:00:00,25,90,3,7417777,NaN,0,None
6,41,S,2012-04-30 17:12:48,0,PI00000000041,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,GE CONSUMER INDUSTRIAL ARGEN,0,2012-05-02 07:00:00,2012-05-03 07:00:00,25,90,3,7417778,NaN,0,None
7,42,S,2012-04-30 17:12:48,0,PI00000000042,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,"PACKALL SA,.",0,2012-05-02 07:00:00,2012-05-03 07:00:00,25,90,3,7417779,NaN,0,None
8,43,S,2012-04-30 17:12:48,0,PI00000000043,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,PIERO S.A.I.C...,0,2012-05-02 08:00:00,2012-05-03 08:00:00,25,90,3,7417780,NaN,0,None
9,44,S,2012-04-30 17:12:49,0,PI00000000044,PERMISO DE INGRESO GENERADO DESDE AGENDA WF,None,None,AUTOSAL S.A.,0,2012-05-02 08:00:00,2012-05-03 08:00:00,25,90,3,7417782,NaN,0,None


In [5]:
df_PI.shape

(40, 9)

In [4]:
Q_FLUJO_2 = """
SELECT 
    ID_tipopi,
    CAB.ID PI,
    TIP.DESCRIPCION PI_DESC, 
    MOV.ID_ESTADO,
    EST.DESCRIPCION ESTADO, 
    MOV.STAMP FECHA, 
    ROW_NUMBER() OVER (PARTITION BY CAB.ID ORDER BY MOV.STAMP) AS ORDEN
FROM pi_movimiento MOV
INNER JOIN pi_cabecera CAB ON CAB.ID=ID_CABECERA
INNER JOIN pi_tipo TIP ON TIP.ID =ID_tipopi
INNER JOIN cdj_estado EST ON EST.ID=MOV.ID_ESTADO
WHERE TRUNC(MOV.stamp) = TO_DATE('{current_day}','dd/mm/yyyy')
ORDER BY CAB.ID,MOV.STAMP
"""


df_flujo = retrieve_oracle_data(conn_string_suc, Q_FLUJO_2.format(current_day='15/11/2024'))
df_flujo.head()

,ID_TIPOPI,PI,PI_DESC,ID_ESTADO,ESTADO,FECHA,ORDEN
0,19,1512921,PALLETS Y ENVASES,15,PENDIENTE DE ARRIBO,2024-11-15 03:27:44,1
1,19,1512921,PALLETS Y ENVASES,16,ARRIBADO AL PREDIO,2024-11-15 03:28:01,2
2,19,1512921,PALLETS Y ENVASES,43,PENDIENTE INGRESO,2024-11-15 09:09:43,3
3,19,1512921,PALLETS Y ENVASES,18,INGRESADO AL PREDIO,2024-11-15 09:35:45,4
4,19,1512921,PALLETS Y ENVASES,29,MOVIL RETIRADO,2024-11-15 13:39:12,5


In [5]:
df_flujo.shape

(4600, 7)

In [7]:
q = """select * from PI_INTAGENDA"""

df_flujo = retrieve_oracle_data(conn_string_suc, q)
df_flujo.head()

,ID,ACTIVO,STAMP,VERSION,ID_EMPRESA,ID_ALMACEN,ID_DIVISION,FECHA_TURNO,HORA_TURNO,NUMERO_TURNO,FPROCESO,ID_PI,OPERACION
0,6062,S,2012-05-29 18:03:03,0,1,93,1,2012-06-01,2003-01-01 09:00:00,61,2012-05-29 18:07:36,3882.0,1
1,6342,S,2012-05-30 15:16:03,0,1,93,1,2012-06-01,2003-01-01 06:00:00,104,2012-05-30 15:23:54,4060.0,1
2,6343,S,2012-05-30 15:19:31,0,1,93,1,2012-06-01,2003-01-01 07:00:00,105,2012-05-30 15:23:54,4061.0,1
3,6350,S,2012-05-30 15:24:21,0,1,93,1,2012-06-01,2003-01-01 08:00:00,106,2012-05-30 15:38:58,4066.0,1
4,6351,S,2012-05-30 15:26:28,0,1,93,1,2012-06-01,2003-01-01 09:00:00,107,2012-05-30 15:38:58,4067.0,1


In [2]:







q= """select 

trunc(sysdate) fecha,
a.cempresa empresa, 
a.calmacen almacen, 
0 czonalma, 
0 pasillo, 
a.cnuanden ubicacion,

case 
    when a.ctipoand = 'P' then 2
    when a.ctipoand = 'A' then 3
    when a.ctipoand = 'I' then 4
    when a.ctipoand = 'K' then 5
    when a.ctipoand = 'C' then 6
end ID_TIPO_ZONA,

case 
    when a.ctipoand = 'P' then 'PULMON'
    when a.ctipoand = 'I' then 'INUTILIZADO'
    when a.ctipoand = 'K' then 'PUNTERA DE RACK'
    when a.ctipoand = 'A' then 'ANDEN'
    when a.ctipoand = 'C' then 'CONTROL'
end DESC_TIPO_ZONA,

a.cdivisio ID_DIVISION, 
b.descdivi DESCDIVI, 
0 id_tipo_ubicacion, 
' ' DESC_TIPO_UBICACION,

(SELECT wfadmin.estado_anden(a.cnuanden, a.cempresa, a.calmacen ) FROM DUAL) id_estado_ubicacion,
CASE 
WHEN
  (SELECT wfadmin.estado_anden(a.cnuanden, a.cempresa, a.calmacen ) FROM DUAL) = 'I' THEN 'Inutilizada'

ELSE ''
END desc_estado_ubicacion,
0 id_tipo_rack, ' ' desc_tipo_rack 
from f638mand a
left join f802divi b on b.cempresa = a.cempresa and b.cdivisio = a.cdivisio"""

df_retrabajos = retrieve_oracle_data(conn_string_suc, q)
df_retrabajos.head()




,FECHA,EMPRESA,ALMACEN,CZONALMA,PASILLO,UBICACION,ID_TIPO_ZONA,DESC_TIPO_ZONA,ID_DIVISION,DESCDIVI,ID_TIPO_UBICACION,DESC_TIPO_UBICACION,ID_ESTADO_UBICACION,DESC_ESTADO_UBICACION,ID_TIPO_RACK,DESC_TIPO_RACK
0,2024-11-28,1,93,0,0,V23,4,INUTILIZADO,2,SECTOR FRUTA-VERDURA,0,,I,Inutilizada,0,
1,2024-11-28,1,93,0,0,V24,4,INUTILIZADO,2,SECTOR FRUTA-VERDURA,0,,I,Inutilizada,0,
2,2024-11-28,1,93,0,0,V25,4,INUTILIZADO,2,SECTOR FRUTA-VERDURA,0,,I,Inutilizada,0,
3,2024-11-28,1,93,0,0,V26,4,INUTILIZADO,2,SECTOR FRUTA-VERDURA,0,,I,Inutilizada,0,
4,2024-11-28,1,93,0,0,V29,4,INUTILIZADO,2,SECTOR FRUTA-VERDURA,0,,I,Inutilizada,0,


In [3]:
####################################################################################################
from model.model import *
Inv = Inventario()

df, cols_missing = Inv.validate_dataframe(df_retrabajos, drop_columns=True)

print(cols_missing)
df.head()


['id_dia', 'id_empresa', 'id_almacen', 'id_zona', 'id_pasillo', 'id_ubicacion', 'id_tipo_ubicacion', 'id_tipo_zona', 'id_tipo_rack', 'id_pallet', 'id_estado_ubicacion', 'id_division_asignada', 'id_producto', 'id_departamento', 'id_clase', 'id_gran_categoria', 'id_gerente_comercial', 'id_dia_ingreso_pallet', 'id_dia_caducidad_pallet', 'capacidad_ubicacion', 'antiguedad_pallet', 'pallets', 'bultos', 'unidades']
['id_dia', 'id_empresa', 'id_almacen', 'id_zona', 'id_pasillo', 'id_ubicacion', 'id_tipo_ubicacion', 'id_tipo_zona', 'id_tipo_rack', 'id_pallet', 'id_estado_ubicacion', 'id_division_asignada', 'id_producto', 'id_departamento', 'id_clase', 'id_gran_categoria', 'id_gerente_comercial', 'id_dia_ingreso_pallet', 'id_dia_caducidad_pallet', 'capacidad_ubicacion', 'antiguedad_pallet', 'pallets', 'bultos', 'unidades']


,id_tipo_zona,id_tipo_ubicacion,id_estado_ubicacion,id_tipo_rack
0,4,0,I,0
1,4,0,I,0
2,4,0,I,0
3,4,0,I,0
4,4,0,I,0


In [4]:
from model.model import *
Inv = Inventario()
Inv.get_fields()

(['id_dia',
  'id_empresa',
  'id_almacen',
  'id_zona',
  'id_pasillo',
  'id_ubicacion',
  'id_tipo_ubicacion',
  'id_tipo_zona',
  'id_tipo_rack',
  'id_pallet',
  'id_estado_ubicacion',
  'id_division_asignada',
  'id_producto',
  'id_departamento',
  'id_clase',
  'id_gran_categoria',
  'id_gerente_comercial',
  'id_dia_ingreso_pallet',
  'id_dia_caducidad_pallet',
  'capacidad_ubicacion',
  'antiguedad_pallet',
  'pallets',
  'bultos',
  'unidades'],
 ['id_dia',
  'id_empresa',
  'id_almacen',
  'id_zona',
  'id_pasillo',
  'id_ubicacion',
  'id_tipo_ubicacion',
  'id_tipo_zona',
  'id_tipo_rack',
  'id_pallet',
  'id_estado_ubicacion',
  'id_division_asignada',
  'id_producto',
  'id_departamento',
  'id_clase',
  'id_gran_categoria',
  'id_gerente_comercial',
  'id_dia_ingreso_pallet',
  'id_dia_caducidad_pallet',
  'pallets',
  'bultos',
  'unidades'],
 ['capacidad_ubicacion', 'antiguedad_pallet'],
 [{'bi_type': 'attribute', 'data_type': str, 'default': '01-01-1900'},
  {'bi_t